# BERT
- BERT 모델의 개념과 학습 방식에 대해 살펴보자

## BERT (**B**i-directional **E**ncoder **R**epresentations from **T**ransformers)

- BERT는 bi-directional Transformer로 이루어진 언어 모델
- 잘 만들어진 BERT 언어 모델 위에 1개의 classification layer만 부착하여 다양한 NLP task를 수행
- 영어권에서 11개의 NLP task에 대해 sate-of-the-art(SOTA) 달성


## Methods - BERT 모델 학습 데이터


- BERT는 transformer의 incoder만 사용해서 언어를 embedding 함
- 학습 코퍼스 데이터
  - BooksCorpus (800M words)
  - English Wikipedia (2,500M words without lists, tables and headers)
  - 30,000 token vocabulary

- 데이터의 tokenizing
  - **WordPiece** tokenizing  
    He likes playing -> He likes play **##ing**
      - 빈도수가 많이 나오는 character 단위 인데 이런 걸 빈도수에 따라 분리하는 토크나이징 방법을 채택
  - 입력 문장을 tokenizing하고, 그 token들로 'token sequence'를 만들어 학습에 사용
  - 2개의 token sequence가 학습에 사용



# Methods - BERT의 WordPiece tokenizing
- Byte Pair Encoding (BPE) 알고리즘 이용
- **빈도수**에 기반해 단어를 **의미 있는 패턴**(Subword)으로 잘라서 tokenizing

- **BPE의 순서도**
  - 1. **Vocab**을 만들기 위한 **문서의 전처리 과정**  
    기준이 되는 문서를 **Character 단위로 분리**
    - 1. Sentences 
    - 2. Tokenized Sentences
      - Tokenize(with ##)

  - 2. Character 단위로 분리된 문서에서 **가장 많이 등장**하는 **Bi-gram Pair를 찾고 합쳐주는 과정**  
  Iteration을 정해 놓고 **주어진 횟수 만큼** 수행
    - 3. bi-gram pairs
    - 4. best pair -> Tokenized Sentences
      - merge best pair

  - 3. 정의된 Iterationd을 모두 수행 후 문서를 Tokenize 해서 최종으로 **Vocab을 생성**
    - 5. Vocab


- 즉, Sentences를 Tokenizeing하고 바로 Vocab을 만드는 것이 아님
- **반복해서 빈도수 계산을 한 뒤 Vocab 후보를 업데이트 하고 그 후보가 최종적인 Vocab이 된다.**


< 예제 >
- **1. Tokenize**   
경찰정 철창살은 외철창살이고 검찰청 철창살은 쌍철창살이다

  Sentence   
  경찰청 -> 경 찰 청  
  철창살은 -> 철 창 살 은  

  Tokenized Sentence (iter=0)
  경 ##찰 ##청  
  철 ##창 ##살 ##은

  - 같은 글자라도 맨 앞에 나오는 것과 아닌 것에는 차이가 있다고 가정  
  - BERT의 경우 뒷단어에 '##'을 붙여서 구별  
  e.g.) '철' != '##철', '철창살' != '##철창살'


- **2. Building Vocab**

  Tokenized Sentence (iter=0)  
  경 ##찰 ##청
  철 ##창 ##살 ##은  

  Vocab(iter=0) **Vocab 후보!**  
  경 ##찰 ##청
  철 ##창 ##살 ##은

  - Vocab의 생성은 정해진 Iteration을 모두 수행 후 Tokenized Sentence를 Tokenize해서 생성


- **3. Bi-gram Pair Count**

  **Vocab 후보 기준으로**  
  Tokenized Sentence(iter=0)  
  경 ##찰 ##청
  철 ##창 ##살 ##은

  Bi-gram pairs(iter=1)  
  (경, ##찰), (##찰, ##청)  
  (철, ##창), (##창, ##살), (##살, ##은)
  
  - 이렇게 만든 후 빈도수를 확인  
    (경, ##찰) : 1, (##살 ##은) : 2, (##창, ##살) : 4  
  - 그 중 **가장 높은 빈도수**를 가진 **Character bi-gram pair**을 얻을 수 있음 


- **4. Merge Best pair**  

  Best Pair : ##창 ##살 -> **##창살**
    - 이때 ##창 ##살이 많이 나왔다는 것은, 이 두 개의 묶음이 중요하다는 뜻  
    - 그래서 이 두 개를 **하나로 합친다**  
    ##창살

  Tokenized Sentence(iter=0)  
  경 ##찰 ##청  
  철 ##창 ##살 ##은  

  Tokenized Sentence(iter=1)  
  경 ##찰 ##청
  철 **##창살** ##은


- **5. Building Vocab**

  Tokenized Sentences(iter=1)  
  경 ##찰 ##청  
  철 ##창살 ##은  

  **Vocab 후보 업데이트**
  Vocab(iter=1)  
  경 ##찰 ##청  
  철 **##창살** ##은  

  - Vocab의 생성은 정해진 Iteration을 모두 수행 후 Tokenized Sentence를 Tokenize해서 생성
  - 이 과정을 반복. 이때 Best pair가 여러 개여도 하나만 선택(후보의 빈도수가 같으면 가장 먼저 있는 것을 선택함)
  - 만들어진 최종 Vocab들은 Vocab list에 저장



- BPE로 subword로 분리했음에도 불구하고 **vocab에 존재하지 않는 단어**는 **[UNK} 토큰**으로 변환


# Methods - Masking 기법

- 데이터의 tokenizing
  - Masked language model (MLM) : input token을 일정 확률로 masking
  - 학습 데이터는 corpus를 이용하여 학습할 거임
    - 학습 데이터에서 문장을 하나 가져온 뒤, 문장의 첫 번째에는 **[CLS]**, 마지막에는 **[SEP]**를 붙임 

    - 이때 두 번째 문장 corpus의 다음 문장으로 만들어질 수도 있고, 50%의 확률로 다른 곳에서 random하게 가져올 수도 있음
    - 학습을 2가지를 함
      - 1. Next sentence를 예측하는 것으로 BERT가 transformer로 학습이 이루어짐(IsNext or NotNext)
      - 2. mask, 혹은 replace된 단어가 정답을 맞출 수 있도록 학습이 이루어짐
    - 이런 식으로 학습 데이터로 사용된 언어를 이 모델로 표현하게 되는 것



- input에서 단순히 vocab으로 token들이 들어가는 게 아님
  - 1. Token Embeddings : 입력 token에 대해서 Embedding layer를 하나 거침
  - 2. Segment Embeddings : 문장 A 인지, 문장 B 인지를 Embedding layer를 거쳐 
  - 3. Position Embeddings : token이 그대로 들어가면 BERT에서 순서의 정보를 얻을 수 없음. 그래서 Position Embedding을 거쳐 Token 개수에 따라 Position Embedding을 다르게 한 뒤 학습



# Methods

- NLP 실험
  - Sentence pair classification
    - 두 가지 sentence를 넣고 이것들이 어떤 관계를 가지는지를 표현
    - ex) A 문장 다음에 B 문장이 자연스러움을 가지고 있는지?  
    A 문장과 B 문장이 의미상으로 유사한가?
  - Single sentence pair classification
    - 영화 리뷰 댓글을 입력으로 넣고, 긍정인지, 중립인지, 부정인지 판단
  - Question and answering (SQuAD v1.1)
    - 질문과 질문에 대한 답을 포함하고 있는 Paragraph을 넣어 답변을 찾아냄
      - 답변이 text로 나오는 것이 아니라, Paragraph이 tokenizing 된 index 값을 출력한다
  - Single sentence tagging
    - 각각의 token에 대한 출력(정보값)을 만들어냄
       - 예) 개채명이 될 수 있음  
       SK -> 기관명의 시작
       telecom -> 기관명의 끝  
       이처럼 BIO tag(시작, 중간, 끝)를 BERT 구조를 이용하여 학습할 수 있음


# BERT 적용 실험
1. 감정 분석
  - 네이버 영화 리뷰 코퍼스(https://github.com/e9t/nsmc)로 감성 분석 진행
2. QnA 문장 유사도(이진 분류 기반)
  - 디지털 동반자 패러프레이징 질의 문장 데이터를 이용하여 질문-질문 데이터 생성 및 학습
  - Saltlux ADAM 일반 질의 데이터를 이용해 IRQA 모사 데이터 구축  
  500 문장에 대한 BERT semantic search(httpsL//github.com/hanxiao/bert-as-service)
3. QnA 문장 유사도(문장 벡터 기반)
  - Sent2Vec 이용, 500개 입력 문장과 가장 유사한 10개의 문장 후보 추출(top 10개 이내 98% 존재)  
  10개 문장과 입력 문장을 BERT로 유사 유/무 비교(이진분류)
  - ADAM 일반 질의 데이터 중, most similar Top 10 문장들을 이용해 학습 데이터 구축  
  **Sent2Vec + BERT Multi-lingual pretrained model**을 사용하니 정확도 높아짐
3. 관계 추출 실험
 - KAIST가 구축한 Silver data 사용(1명의 전문가가 annotaion)
4. 개체명 인식
  - BERT 모델에서 맨 마지막에 sequence tagging을 위한 layer를 쌓아서 구현  
  공개된 다국어 모델 활용, 음절 단위로 입력을 하도록 구현(https://github.com/kyzhouhzau/BERT-NER)
5. 기계독해 실험
  - LG CNS가 공개한 한국어 QA 데이터 셋, KorQuAD(https://korquad.github.io/)  
  어절 단위로 tokenizing 하면 '은', '의'와 같이 정답에 조사가 많이 붙어있어서 EM이 낮게 나옴. ex) 이순신의    
    **음절(character) 단위로 tokenizing 하니 EM과 F1이 올랐음**

